In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import time

# Manipulate Page

In [2]:
driver_location = "/Users/davidmanspeaker/Downloads/chromedriver"
driver = webdriver.Chrome(executable_path=driver_location)
driver.get("http://www.google.com")
driver.find_element_by_class_name('gb_Ne').click()
inputElement = driver.find_element_by_id("Email")
inputElement.send_keys('david4cider@gmail.com')
inputElement.send_keys(Keys.ENTER)

time.sleep(3)

inputElement = driver.find_element_by_id("Passwd")
inputElement.send_keys('Lifeisgood!')
inputElement.send_keys(Keys.ENTER)

# Set Up Google Scrape

In [3]:
%system GoogleScraper -m http -p 100 -n 50 -q "should site:reddit.com/r/AskReddit/" --output-filename reddit.json

['2016-09-11 19:03:30,052 - GoogleScraper.caching - INFO - 2521 cache files found in .scrapecache/',
 '2016-09-11 19:03:30,052 - GoogleScraper.caching - INFO - 100/100 objects have been read from the cache. 0 remain to get scraped.']

# Open the file and Get Results

In [4]:
with open("reddit.json", 'r') as f:
    data  = json.load(f)
    
all_results = []
for d in data:
    all_results.extend(d['results'])

# Inspect html

In [7]:
questions = []
for result in all_results:
    link = result['link']
    driver.get(link)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    question_f = soup.find("a", { "class" , "title may-blank" })
    comments_f = soup.find("a", { "class" , "bylink comments may-blank" })
    
    if question_f is not None:
        question = question_f.text
        comments = -1
        if comments_f is not None:
            comments = comments_f.text
        
        questions.append([link, question.strip(), comments])
    else:
        questions.append([link, None, None])

KeyboardInterrupt: 

# Output Results as CSV

In [8]:
scrapeDF = pd.DataFrame(questions, columns=['Link', 'Question', 'Answers/Comments'])
scrapeDF.to_csv('Results/Reddit Scrape Results_Should' + time.strftime("%Y%m%d-%H%M%S") + '.csv', index=False)